## Configuration

### Proxy Role
AWS Role - 'arn:aws:iam::946429944765:role/irsa-user-sa-proxy-role'
Trust Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::11111:oidc-provider/oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260xxx"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260xxx:aud": "sts.amazonaws.com",
                    "oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260xxx:sub": [
                        "*:domino-compute:integration-test",
                        "*:domino-compute:john-doe",
                        "*:domino-compute:jane-doe"
                    ]
                }
            }
        }
    ]
}
```
Permission Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sts:AssumeRole",
            "Resource": "arn:aws:iam::11111:role/irsa-user-sa-asset-role"
        }
    ]
}
```

### Asset Role
AWS Role - 'arn:aws:iam::11111:role/irsa-user-sa-asset-role'
Trust Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::11111:role/irsa-user-sa-proxy-role"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```
Permission Policy - Undefined for this demo. But can be anything the admin allows, say listing of a S3 bucket.


In [ ]:
import boto3
import os

In [ ]:
os.environ['AWS_ROLE_ARN']='arn:aws:iam::946429944765:role/irsa-user-sa-proxy-role'

In [ ]:
sts = boto3.client('sts')
sts.get_caller_identity()

In [ ]:
response = sts.assume_role(
    RoleArn="arn:aws:iam::946429944765:role/irsa-user-sa-asset-role",
    RoleSessionName="irsa-user-sa-asset-role"
)
print(response['AssumedRoleUser'])